In [92]:
import sys
sys.path.append("../../")
from taxcalc import *
import pandas as pd
import numpy as np

In [152]:
records_x = Records("../../puf.csv")
params_x = Policy()
records_mid = Records("../../puf.csv")
params_mid = Policy()
records_high = Records("../../puf.csv")
params_high = Policy()

In [153]:
reform_rates = {
        2016: {
            '_AMT_trt1': [.0],
            '_AMT_trt2': [.0]
           },}


reform_rates_high = {
    2016: {
            '_II_rt5': [.286],
            '_II_rt6': [.286],
            '_II_rt7': [.286],
            '_AMT_trt1': [.0],
            '_AMT_trt2': [.0]
           },}

reform_rates_mid = {
    2016: {
            '_II_brk2': [[65005, 130010, 65005, 88180,0,0]],
            '_AMT_trt1': [.0],
            '_AMT_trt2': [.0]
           },}

params_mid.implement_reform(reform_rates_mid)
params_high.implement_reform(reform_rates_high)
params_x.implement_reform(reform_rates)

In [154]:
calc_x = Calculator(params_x, records_x)
calc_mid = Calculator(params_mid, records_mid)
calc_high = Calculator(params_high, records_high)
for i in range(3):
    calc_x.increment_year()
    calc_mid.increment_year()
    calc_high.increment_year()


You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.
You loaded data for 2009.
Your data have been extrapolated to 2013.


In [155]:
print calc_x.current_year == 2016

True


In [156]:
calc_x.calc_all()
calc_mid.calc_all()
calc_high.calc_all()

### Costs of the two tax cuts are very similar

In [158]:
cost_mid = ((calc_mid.records._combined - calc_x.records._combined)*calc_mid.records.s006).sum()
cost_high = ((calc_high.records._combined - calc_x.records._combined)*calc_high.records.s006).sum()
print cost_mid
print cost_high

-101735788191.0
-102137246528.0


In [159]:
mtr_fica_x, mtr_iit_x, mtr_combined_x = calc_x.mtr()
mtr_fica_mid, mtr_iit_mid, mtr_combined_mid = calc_mid.mtr()
mtr_fica_high, mtr_iit_high, mtr_combined_high = calc_high.mtr()

In [160]:
avg_mtr_combined_x = ((mtr_combined_x * calc_x.records.c00100 * calc_x.records.s006).sum()/
                      (calc_x.records.c00100 * calc_x.records.s006).sum())
avg_mtr_combined_mid = ((mtr_combined_mid * calc_mid.records.c00100 * calc_mid.records.s006).sum()/
                      (calc_mid.records.c00100 * calc_mid.records.s006).sum())
avg_mtr_combined_high = ((mtr_combined_high * calc_high.records.c00100 * calc_high.records.s006).sum()/
                      (calc_high.records.c00100 * calc_high.records.s006).sum())

print avg_mtr_combined_x
print avg_mtr_combined_mid
print avg_mtr_combined_high

0.36052332838
0.339640431591
0.33955243425


### Average marginal tax rate reductions of the two tax cuts are very close

In [161]:
print avg_mtr_combined_mid - avg_mtr_combined_x
print avg_mtr_combined_high - avg_mtr_combined_x

-0.0208828967887
-0.0209708941302


### But 55% of expanding the 15% bracket is spent on inframarginal tax reductions

In [162]:
inframarginal_mid = (((calc_x.records._combined-calc_mid.records._combined)*calc_x.records.s006)
                        [(mtr_combined_x==mtr_combined_mid)&(calc_x.records._combined>calc_mid.records._combined)].sum())
inframarginal_high = (((calc_x.records._combined-calc_high.records._combined)*calc_x.records.s006)
                        [(mtr_combined_x==mtr_combined_high)&(calc_x.records._combined>calc_high.records._combined)].sum())

In [163]:
print inframarginal_mid/cost_mid
print inframarginal_high/cost_high

-0.549052709702
-0.0
